# Exploratory Data Analysis 
##         -- Children's Imagined Future and its association with socioeconomic status

## Background 

“Imagine you are now 25 years old. Write about the life you are leading, your interests, your home life and your work at the age of 25”. As a part of a longitudinal study on individuals born in 1958, over 10,000 11-year-old children in the UK were asked to write an essay given the instruction above (Center for Longitudinal Studies (CLS) | 1958 National Child Development Study). Their families’ demographic information, including socioeconomic status (SES) related items (e.g., father’s occupation), was recorded. At the age of 42, they participated in another sweep of the study, which collected information including their income range. 

One study examined this corpus and themes of children’s imagined future. With selected sample and a qualitative approach, it identified a wide range of themes, including marriage, family members, occupation, money, domestic life, vacations, leisure activities and animals. Although the children rarely described their future in terms of status, researchers discovered that high SES children are more likely to write about imaginations characterizing high-prestige lifestyles (e.g. large house, fancy vehicle and overseas vacation) (Elliott, 2010).

So far, no existing literature using this corpus has investigated this relationship of SES and children’s writing with a machine learning/NLP approach. Thus, I am here to fill the gap and discover new knowledge:). 

## Data 

We (my collaborator Dr. Holly Engstrom) obtained the longitudinal data at CLS' data access application portal. We first selected the age-11 and age-42 waves as our data, which contain information on:
1. children's essays
2. children's father's occupation
3. whether the family experienced financial hardship
4. weekly income at the age of 42.  

We had put enormous effort to clean and preprocess the data, including manually correcting all the misspellings in the essays. The data after processing is saved in full_data.csv under the **[current repository](https://github.com/h-karyn/ImaginedFutureAndSocioeconomics)**. 

## Data Overview    

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read data 
data_df = pd.read_csv('full_data.csv') 
data_df.shape

(18567, 13)

In [3]:
# get an overview of the top-n rows/samples
data_df.head()

,ID,file_name,original,corrected,misspellings,readable?,RA_name,completed?,word_count,father,mother,financial,pay_per_week
0,N10001N,B771-780,Today is my 25th birthday and I am in hospital...,Today is my 25th birthday and I am in hospital...,10.0,yes,Anika,yes,306.0,2.0,-1.0,2.0,237.25
1,N10002P,J1-10,The dog pounded up at me when i reached for he...,The dog pounded up at me when i reached for he...,9.0,yes,Andrea,yes,200.0,-1.0,-1.0,-1.0,NaN
2,N10003Q,J1-10,I would get married and be a swimming instruct...,I would get married and be a swimming instruct...,4.0,yes,Andrea,yes,268.0,3.0,6.0,1.0,NaN
3,N10004R,A821-830,When I am 25 years old. I hop that I will be m...,When I am 25 years old. I hope that I will be ...,23.0,yes,Manson,yes,169.0,2.0,8.0,2.0,221.25
4,N10005S,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,6.0,2.0,NaN


## Data Cleaning

Since column file_name, original, misspellings, RA_name, completed?, word_count, and mother are not revalent to our analysis, I will drop those columns. 

In [4]:
# drop irrelevant columns
data_df = data_df.drop(['file_name', 'original', 'misspellings', 'word_count','RA_name', 'completed?', 'mother'], axis=1)
data_df.head(3)

,ID,corrected,readable?,father,financial,pay_per_week
0,N10001N,Today is my 25th birthday and I am in hospital...,yes,2.0,2.0,237.25
1,N10002P,The dog pounded up at me when i reached for he...,yes,-1.0,-1.0,NaN
2,N10003Q,I would get married and be a swimming instruct...,yes,3.0,1.0,NaN


In [5]:
# drop rows with missing values
data_df = data_df.dropna()

data_df.shape

(4843, 6)

Clean the 'readable?' column

In [6]:
# turn 'readable?' to lowercase
data_df['readable?'] = data_df['readable?'].str.lower()

# check the unique values
print(data_df['readable?'].unique())

# replace 'yexs' and 'yea' with 'yes'
data_df['readable?'] = data_df['readable?'].replace(['yexs', 'yea'], 'yes')

# check the unique values again
print(data_df['readable?'].unique())

# drop rows with 'no'
data_df = data_df[data_df['readable?'] == 'yes']

data_df.shape

['yes' 'no' 'yexs' 'yea']
['yes' 'no']


(4720, 6)

In [7]:
# preview the data
data_df.head(3) 

,ID,corrected,readable?,father,financial,pay_per_week
0,N10001N,Today is my 25th birthday and I am in hospital...,yes,2.0,2.0,237.25
3,N10004R,When I am 25 years old. I hope that I will be ...,yes,2.0,2.0,221.25
12,N10013S,Now that I am 25 and I have left college I thi...,yes,2.0,2.0,729.00


## Data Preprocessing

In this data set, we label high SES as 1 and low SES as 2: * father’s occupation is coded as: 1 - non-manual; 2 - manual; * financial hardship is coded as: 1 - No; 2 - Yes;

Combination of father’s occupation and financial hardship: * 1 - father had a non-manual job & no financial hardship. * 2 - father had a manual job & no financial hardship. * 3 - father had a non-manual job & financial hardship. * 4 - father had a manual job & financial hardship.

In [8]:
# Define a function to apply the conditions
def combine_columns(row):
    if row['father'] == 1 and row['financial'] == 1:
        return 1
    elif row['father'] == 2 and row['financial'] == 1:
        return 2
    elif row['father'] == 1 and row['financial'] == 2:
        return 3
    elif row['father'] == 2 and row['financial'] == 2:
        return 4
    else:
        return None 

In [9]:
# Apply the function to each row
data_df['family_ses'] = data_df.apply(combine_columns, axis=1)

# Convert columns to categorical data type
data_df['father'] = data_df['father'].astype('category')
data_df['financial'] = data_df['financial'].astype('category')
data_df['family_ses'] = data_df['family_ses'].astype('category')

#preview the data
data_df.head(3)

,ID,corrected,readable?,father,financial,pay_per_week,family_ses
0,N10001N,Today is my 25th birthday and I am in hospital...,yes,2.0,2.0,237.25,4.0
3,N10004R,When I am 25 years old. I hope that I will be ...,yes,2.0,2.0,221.25,4.0
12,N10013S,Now that I am 25 and I have left college I thi...,yes,2.0,2.0,729.00,4.0


In [10]:
# check the datatype of each column
data_df.dtypes

ID                object
corrected         object
readable?         object
father          category
financial       category
pay_per_week     float64
family_ses      category
dtype: object

Till this point, we have:

1. categorical variable `family_ses`, which will be the target variable for **supervised learning - classification**
2. numerical variable `income`, which will be the target variable for **supervised learning - regression**
3. text variable `essay`, which will used for **unsupervised learning - topic modeling**. Also, it will serve as the input for the supervised learning models.

Next, we need to preprocess the text data.

## Text Preprocessing

In [11]:
# tokenize and lemmatize the text
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer

# download the wordnet dictionary
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/mhhuang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mhhuang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/mhhuang/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mhhuang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
corpus = data_df['corrected'].tolist()
corpus_lem = []
lemmatizer = WordNetLemmatizer()
for text in corpus:
    word_list = nltk.word_tokenize(text)
    word_lem = [lemmatizer.lemmatize(tok) for tok in word_list]
    corpus_lem.append(' '.join(word_lem))

In [13]:
# remove all the numbers and punctuations
import re
corpus_lem = [re.sub(r'[^a-zA-Z]', ' ', text) for text in corpus_lem]

In [14]:
vectorizer = CountVectorizer(min_df=5, max_df=0.8,ngram_range=(1,1),binary=False,stop_words='english')
X = vectorizer.fit_transform(corpus_lem)

In [15]:
feature_names = vectorizer.get_feature_names_out()

Find out the top-10 most frequent features

In [16]:
def get_topn_features(X, feature_names, topn=10):
    """
    Inputs:
        X: feature matrix
        feature_names: extracted features during vectorization
        topn: the number of most frequent features to return
    Outputs:
        topn most frequent features and their frequency
    """
    feature_ct = np.asarray(np.sum(X, axis=0)).reshape(-1)

    feature_freq = []
    
    for i in np.argsort(feature_ct)[::-1][:topn]:
        feature_freq.append({'feature':feature_names[i], 'frequency':feature_ct[i]})
    
    return pd.DataFrame(feature_freq)

In [17]:
get_topn_features(X, feature_names, topn=30)

,feature,frequency
0,like,7322
1,work,6905
2,wa,5742
3,child,4838
4,home,4309
5,year,4087
6,house,3585
7,time,3535
8,job,3262
9,got,2932
